In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


1.   **Import anything**



In [ ]:
import pandas as pd
import numpy as np
import cv2
import shutil
import json
import sys
import matplotlib.pyplot as plt
import shutil
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm
import time
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import joblib
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3, EfficientNetB4

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = torch.device('cpu')

Using CUDA


2.   **Setting directories**



In [ ]:
# 이 부분을 추가로 주어지는 20개의 이미지가 담겨 있을 파일의 경로도 추가해서 돌리면 됨.
print(os.listdir("/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023"))

labeledimg_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑"
name_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품명_labeling용.txt"

train_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑TrainTestFolder/train"
test_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/상품매핑TrainTestFolder/test"

backbone_dir="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/backbonefolder"

['frozen_inference_graph.pb', '상품명_labeling용.txt', 'labeled_data.ndjson', '상품매핑', 'reference.gdoc', '상품매핑TrainTestFolder', '.ipynb_checkpoints', 'backbonefolder', 'mask_rcnn_inception_v2_coco_2018_01_28.pbtxt', 'validation', 'image_preprocessing', 'validation1_5.json', 'validation_set6_9.json', 'validation_set10.json', 'detectron2_addfunction(2).ipynb', 'detectron2_addfunction_최종.ipynb', 'Validation TrainTestFolder', 'detectron2_ipynb의_사본.ipynb', 'detectron2_addfunction_최종(2).ipynb', 'detectron2_addfunction_최종(2) (1) (1).ipynb', 'Validation _Train_Test', '상품매핑PreprocessedFolder', 'Validation_TrainTest', 'detectron2_addfunction_최종(2) (1).ipynb', 'merged1_9.json', 'validation1_10.json', 'output_directory', 'Json', 'reference_YOLOv5.ipynb', 'merged1_6.json', 'output.json', 'output1_6.json', 'save_result.ipynb', 'merged8_9.json', 'OutputModel', 'NewImage', 'image_preprocessing_code.ipynb', 'yolodata.ipynb', 'yolo_data.zip', 'yolo.yaml', 'yolo_lo

In [ ]:
print('총 데이터의 갯수 : {}개'.format(len(os.listdir(labeledimg_dir))))
print('Train 데이터의 갯수 : {}개'.format(len(os.listdir(train_dir))))
print('Test 데이터의 갯수 : {}개'.format(len(os.listdir(test_dir))))

총 데이터의 갯수 : 598개
Train 데이터의 갯수 : 418개
Test 데이터의 갯수 : 180개


In [ ]:
classes = []
for filename in os.listdir(labeledimg_dir):
    if filename.endswith(".jpg"):
        class_name = filename.split(')')[0]
        if class_name not in classes:
            classes.append(class_name)

print(f"Number of classes: {len(classes)}")
print(f"Classes: {classes}")


Number of classes: 100
Classes: ['99', '98', '97', '96', '95', '93', '92', '90', '91', '89', '87', '88', '86', '85', '84', '82', '81', '80', '83', '79', '77', '74', '75', '76', '73', '72', '70', '69', '68', '67', '65', '63', '64', '62', '61', '60', '59', '58', '57', '56', '54', '55', '53', '52', '50', '51', '49', '47', '48', '45', '46', '43', '44', '42', '40', '38', '41', '37', '35', '34', '31', '32', '30', '26', '29', '25', '24', '21', '23', '19', '20', '17', '18', '16', '15', '13', '14', '11', '110', '114', '113', '109', '111', '108', '107', '105', '103', '102', '100', '101', '09', '04', '05', '10', '08', '07', '06', '02', '03', '01']


3.   **BackBone model for Extraction**

*   **아래의 클래스들은 Detectron2에 backbone model로 사용할 수 있는 모델 클래스입니다.**
*   **기존의 backbone은 VGG16이고 이를 제외한 ResNet, GoogLeNet, ResGoogLeNet을 custom backbone으로 생성하였습니다.**


In [ ]:
# getpath 함수 설명
# 파라미터 : path(기본 경로에 join으로 찾을 경로주소)
# 예를 들어 path 파라미터로 상품매핑TrainTestFolder/train 을 입력할 수 있습니다.

original_path="/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023"
afterpath = 'jsonnew_1.json'

def getpath(path) :
    result_path = os.path.join(original_path, path)
    return result_path

In [ ]:
# "/content/drive/MyDrive/CJ대한통운 미래기술챌린지 2023/backbonefolder" 경로에서 ModelClass 내부 클래스를 호출하는 코드

sys.path.append(backbone_dir)
from ModelClass import ResNet, GoogLeNet, ResGoogLeNet, EfficientNetB4, MyDataset, train, test
sys.path.append(getpath('image_preprocessing'))
from Splitandmakedir import split_data, make_dir, putindata, loadinzip, removeinFolder

In [ ]:
resnet50 = ResNet()
gLN = GoogLeNet()
resgLN = ResGoogLeNet()
#eff = EfficientNetB4()
if torch.cuda.is_available():
    resnet50.cuda()
    gLN.cuda()
    resgLN.cuda()
    #eff.cuda()

4.   **Code BaseLine for Extraction**

*   **Train과 Test 정확도는 상품매핑 이미지데이터셋을 통해 Split한 데이터로 계산된 것으로 실제 모델에 적합 전 대략적인 값을 파악하기 위해 실행하였습니다. 그리고 GPU 시간은 개인차가 있을 수 있습니다.**
*   **참고로 Pretrained model만의 정확도와 Faster R-CNN, Masked R-CNN 모델에 적합 시 정확도 차이는 대략적으로 3~5% 차이가 있다고 합니다.**
*   **Pretrained model 평균정확도 비교 : Resnet50 > ResGoogLeNet > GoogLeNet**
*   **Pretrained model 연산속도 비교 : GoogLeNet > ResGoogLeNet > Resnet50**

In [ ]:
# loadinzip 함수 설명
# zip폴더 내부의 폴더와 파일들을 ratio로 Train/Test 나누는 코드입니다.

zippath = loadinzip(src_dir=getpath('augmentation'),)
for path in zippath:
  split_data(
      src_dir=getpath(path),
      train_dir=getpath('상품매핑TrainTestFolder/train'),
      test_dir=getpath('상품매핑TrainTestFolder/test'),
      ratio=0.7)


In [ ]:
# train/test를 데이터셋에 등록하고 로드하는 방법입니다.

dataset_train = MyDataset(labeledimg_dir=getpath('상품매핑TrainTestFolder/train'))
dataloader_train = DataLoader(dataset=dataset_train, batch_size=32)
dataset_test = MyDataset(labeledimg_dir=getpath('상품매핑TrainTestFolder/test'))
dataloader_test = DataLoader(dataset=dataset_test, batch_size=32)

In [ ]:
%%time
model = resnet50
criterion = torch.nn.CrossEntropyLoss() # 손실 함수를 정의합니다.
optimizer = torch.optim.Adam(model.parameters()) # 최적화 함수를 정의합니다.
for epoch in range(10):
    tr_time, train_acc = train(model, dataloader_train, criterion, optimizer) # 모델을 학습시키고, 학습 정확도를 얻습니다.
    te_time, test_acc = test(model, dataloader_test) # 모델을 평가하고, 테스트 정확도를 얻습니다.
    print(f"\nEpoch {epoch+1}, Train Accuracy: {train_acc:.2f}%, Train Time: {tr_time:.2f}초, Test Accuracy: {test_acc:.2f}%, Test Time: {te_time:.2f}초") # 정확도 값을 출력합니다.

Using CUDA


Description Training: 100%|=========================================| 14/14 [00:19<00:00,  1.39s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:06<00:00,  1.15s/it]



Epoch 1, Train Accuracy: 0.06%, Train Time: 19.52초, Test Accuracy: 0.13%, Test Time: 6.94초
Using CUDA


...


Epoch 9, Train Accuracy: 1.00%, Train Time: 19.01초, Test Accuracy: 0.60%, Test Time: 7.35초
Using CUDA


Description Training: 100%|=========================================| 14/14 [00:19<00:00,  1.37s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:06<00:00,  1.14s/it]


Epoch 10, Train Accuracy: 1.00%, Train Time: 19.24초, Test Accuracy: 0.60%, Test Time: 6.86초
CPU times: user 4min 3s, sys: 7.17 s, total: 4min 10s
Wall time: 4min 23s


In [ ]:
%%time
model = gLN
criterion = torch.nn.CrossEntropyLoss() # 손실 함수를 정의합니다.
optimizer = torch.optim.Adam(model.parameters()) # 최적화 함수를 정의합니다.
for epoch in range(10):
    tr_time, train_acc = train(model, dataloader_train, criterion, optimizer) # 모델을 학습시키고, 학습 정확도를 얻습니다.
    te_time, test_acc = test(model, dataloader_test) # 모델을 평가하고, 테스트 정확도를 얻습니다.
    print(f"\nEpoch {epoch+1}, Train Accuracy: {train_acc:.2f}%, Train Time: {tr_time:.2f}초, Test Accuracy: {test_acc:.2f}%, Test Time: {te_time:.2f}초") # 정확도 값을 출력합니다.

Using CUDA


Description Training: 100%|=========================================| 14/14 [00:29<00:00,  2.14s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:50<00:00,  8.42s/it]



Epoch 1, Train Accuracy: 0.07%, Train Time: 29.96초, Test Accuracy: 0.05%, Test Time: 50.52초
Using CUDA


...


Epoch 9, Train Accuracy: 1.00%, Train Time: 16.63초, Test Accuracy: 0.53%, Test Time: 7.50초
Using CUDA


Description Training: 100%|=========================================| 14/14 [00:16<00:00,  1.20s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:07<00:00,  1.26s/it]


Epoch 10, Train Accuracy: 1.00%, Train Time: 16.77초, Test Accuracy: 0.54%, Test Time: 7.56초
CPU times: user 3min 38s, sys: 8.95 s, total: 3min 47s
Wall time: 4min 54s


In [ ]:
%%time
model = resgLN
criterion = torch.nn.CrossEntropyLoss() # 손실 함수를 정의합니다.
optimizer = torch.optim.Adam(model.parameters()) # 최적화 함수를 정의합니다.
for epoch in range(20):
    tr_time, train_acc = train(model, dataloader_train, criterion, optimizer) # 모델을 학습시키고, 학습 정확도를 얻습니다.
    te_time, test_acc = test(model, dataloader_test) # 모델을 평가하고, 테스트 정확도를 얻습니다.
    print(f"\nEpoch {epoch+1}, Train Accuracy: {train_acc:.2f}%, Train Time: {tr_time:.2f}초, Test Accuracy: {test_acc:.2f}%, Test Time: {te_time:.2f}초") # 정확도 값을 출력합니다.

Using CUDA


Description Training: 100%|=========================================| 14/14 [00:19<00:00,  1.42s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:06<00:00,  1.09s/it]



Epoch 1, Train Accuracy: 0.15%, Train Time: 19.91초, Test Accuracy: 0.23%, Test Time: 6.56초
Using CUDA


...


Epoch 19, Train Accuracy: 0.93%, Train Time: 18.69초, Test Accuracy: 0.35%, Test Time: 7.70초
Using CUDA


Description Training: 100%|=========================================| 14/14 [00:18<00:00,  1.34s/it]


Using CUDA


Description Testing: 100%|============================================| 6/6 [00:07<00:00,  1.32s/it]


Epoch 20, Train Accuracy: 0.87%, Train Time: 18.83초, Test Accuracy: 0.28%, Test Time: 7.91초
CPU times: user 8min 6s, sys: 16.1 s, total: 8min 22s
Wall time: 8min 47s


6.   **Make backbone.h5 in folder**

In [ ]:
def modelfunction() :
  backbonemodel = [resnet50, gLN, resgLN]
  return backbonemodel

In [ ]:
# 모델을 저장합니다.
backbone_dir = getpath('backbonefolder')
backbonemodel = modelfunction()

if not os.path.exists(backbone_dir):
  os.makedirs(backbone_dir)
for model in backbonemodel:
  if model == resnet50:
    backbonename = 'ResNet'
  elif model == gLN:
    backbonename = 'GoogLeNet'
  else:
    backbonename = 'ResGoogLeNet'
  path = os.path.join(backbone_dir, f'backbonemodel-{backbonename}.h5')
  joblib.dump(model, path)